In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns

import nltk
from nltk.tokenize import (sent_tokenize, TreebankWordTokenizer, WhitespaceTokenizer)
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
from nltk.corpus import stopwords

from collections import Counter
from operator import itemgetter

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import re

# from sqlalchemy import create_engine

from sklearn.naive_bayes import GaussianNB, BernoulliNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import (accuracy_score, classification_report,confusion_matrix, precision_score, 
                             recall_score, f1_score, roc_curve, roc_auc_score, average_precision_score, 
                             precision_recall_curve, auc)

from sklearn.cross_validation import train_test_split 
from sklearn.model_selection import KFold  
from sklearn.preprocessing import StandardScaler 

from sklearn.base import TransformerMixin

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

from xgboost import XGBClassifier

import datetime
import warnings
import os

warnings.filterwarnings("ignore")

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# helper functions
def sentence_tokenizer(text):
    sentences = sent_tokenize(text)
    return sentences

def whitespace_tokenizer(sentences):
    listy = []
    tokenizer = WhitespaceTokenizer()
    for i in list(range(0,len(sentences))):
        tokenized = tokenizer.tokenize(sentences[i])
        listy.append(tokenized)
    return listy

def polarity(sentences):
    listy = []
    for i in list(range(0,len(sentences))):
        pol = TextBlob(sentences[i]).polarity
        listy.append(pol)
    return np.min(listy), np.max(listy), np.mean(listy),listy

def stemmer(text):
    stemmer = PorterStemmer()
    listy = []
    for word in TextBlob(text).words:
        listy.append(stemmer.stem(word))
    return listy

def token_clean(text):
    text = text.replace('\n',' ')
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    text = text.lower().split()
    return text

In [3]:
df = pd.read_pickle('../data/toxictrain.pkl')
print(df.shape)
df.head()

(32450, 16)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,rating,token_clean,sent_token,polarity_sent_token,word_count,polarity_min,polarity_max,polarity_mean
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,4,"[cocksucker, before, you, piss, around, on, my...",[COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK],"(0.0, 0.0, 0.0, [0.0])",8,0.000000,0.000000,0.000000
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,1,"[hey, what, is, it, talk, what, is, it, an, ex...","[Hey... what is it..\n@ | talk ., What is it.....","(-0.0333333333333, 0.366666666667, 0.111111111...",50,-0.033333,0.366667,0.111111
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,1,"[bye, dont, look, come, or, think, of, comming...","[Bye!, Don't look, come or think of comming ba...","(0.0, 0.0, 0.0, [0.0, 0.0, 0.0])",10,0.000000,0.000000,0.000000
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,4,"[you, are, gay, or, antisemmitian, archangel, ...","[You are gay or antisemmitian?, Archangel WHit...","(-1.0, 0.416666666667, -0.0359375, [0.41666666...",113,-1.000000,0.416667,-0.035937
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,3,"[fuck, your, filthy, mother, in, the, ass, dry]","[FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!]","(-0.427777777778, -0.427777777778, -0.42777777...",8,-0.427778,-0.427778,-0.427778


In [4]:
df.sum(axis=0,numeric_only=True)

toxic            1.529400e+04
severe_toxic     1.595000e+03
obscene          8.449000e+03
threat           4.780000e+02
insult           7.877000e+03
identity_hate    1.405000e+03
rating           3.509800e+04
word_count       1.959182e+06
polarity_min    -5.685087e+03
polarity_max     5.303854e+03
polarity_mean   -3.346247e+02
dtype: float64

In [5]:
df.groupby('rating').nunique()['id'] # class imbalance issue

rating
0    16225
1     6360
2     3480
3     4209
4     1760
5      385
6       31
Name: id, dtype: int64

In [6]:
# downsampling
# df_t = df[df['rating']>0]
# df_nt = df[df['rating']==0]

In [7]:
df1 = df[['comment_text','toxic']]
X = df1['comment_text']
y = df1['toxic']
print(X.shape, y.shape)

(32450,) (32450,)


In [8]:
count_vect = CountVectorizer()
X_cv = count_vect.fit_transform(df1.comment_text)
X_cv.shape

(32450, 66472)

In [9]:
# X = X.toarray()

In [10]:
# X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y, test_size=0.3,random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42, stratify=y)

k_range = list(range(1, 101))
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

(22715,) (22715,)
(9735,) (9735,)


In [ ]:
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [ ]:
# Define models to test
model_list = [['GaussianNB', GaussianNB()], 
                ['BernoulliNB', BernoulliNB(alpha=0.5)], 
                ['MultinomialNB', MultinomialNB(alpha=0)],
                ['DecisionTree', DecisionTreeClassifier(criterion = 'gini', max_depth=2, splitter='best')], 
                ['KNN', KNeighborsClassifier(27)], 
                ['RandomForest', RandomForestClassifier(max_features='auto',n_estimators=1000)], 
                ['GradientBoost', GradientBoostingClassifier(n_estimators=500)],
                ['AdaBoost', AdaBoostClassifier(algorithm='SAMME.R',
                      base_estimator=DecisionTreeClassifier(criterion = 'gini', max_depth=2, splitter='best'),
                      n_estimators=2)],
                ['XGBoost', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
                       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=6,
                       min_child_weight=11, missing=-999, n_estimators=500, nthread=4,
                       objective='reg:linear', reg_alpha=0, reg_lambda=1,
                       scale_pos_weight=1, seed=0, silent=1, subsample=0.8)],
                ['LogisticRegression', LogisticRegression(C=1000, penalty='l1')],          
                ['SVM', SVC(C=10,gamma=0.001,probability=True)]] 

# Calculate metrics for each model
roc = {}
results_dict = {}
for model in model_list:
    
    model_name = model[0]
    model = model[1]
    pipe = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('to_dense', DenseTransformer()), ('pipe', model)])

    accuracy = []
    precision_1 = []
    precision_0 = []
    recall_1 = []
    recall_0 = []
    f1_1 = []
    f1_0 = []
    auc = []
        
    # Perform K-Fold CV and calculate metrics for each fold
    kf = KFold(5, random_state=42, shuffle=True) 
    for train_idx, test_idx in kf.split(X, y=y):
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        accuracy.append(accuracy_score(y_test, y_pred))
        precision_1.append(precision_score(y_test, y_pred ,pos_label=1))
        precision_0.append(precision_score(y_test, y_pred ,pos_label=0))
        recall_1.append(recall_score(y_test, y_pred, pos_label=1))
        recall_0.append(recall_score(y_test, y_pred, pos_label=0))
        f1_1.append(f1_score(y_test, y_pred, pos_label=1))
        f1_0.append(f1_score(y_test, y_pred, pos_label=0))
        auc.append(roc_auc_score(y_test, y_pred))
        
    # Calculate mean metric across K-folds
    mean_accuracy = np.mean(accuracy)
    mean_precision_1 = np.mean(precision_1)
    mean_precision_0 = np.mean(precision_0)
    mean_recall_1 = np.mean(recall_1)
    mean_recall_0 = np.mean(recall_0)
    mean_f1_1 = np.mean(f1_1)
    mean_f1_0 = np.mean(f1_0)
    mean_auc = np.mean(auc)
    
    # Capture TPR and FPR from last fold for plotting
    y_score = pipe.predict_proba(X_test)[:,1]
    roc[model_name] = roc_curve(y_test, y_score), mean_auc
    results_dict[model_name] = {"accuracy": mean_accuracy, "precision_s": mean_precision_1, "precision_f": mean_precision_0, "recall_s": mean_recall_1, "recall_f": mean_recall_0, "f1_s": mean_f1_1, "f1_f": mean_f1_0, "auc": mean_auc}
    
    # Print formatted results
    print(model)
    print('\t==============================')
    print('\tAccuracy:', mean_accuracy)
    print('\tAUC:', mean_auc)
    print('\n')
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred)) 

In [ ]:
# Plot the ROC curve from the last K-Fold split
fig, ax = plt.subplots(figsize=(10, 10))

# Plot 50-50 Line
ax.plot([0,1],[0,1], ls='--', color='k', label='50-50')

# Plot Classifier ROC Curves
for key, value in roc.items():
    label = '{}, AUC: {}%'.format(key, round(100*value[1],1))
    ax.plot(roc[key][0][0], roc[key][0][1], label=label)
    
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
ax.set_title('ROC Curve - All Models',fontweight='bold',fontsize=15)
ax.legend(loc='best')
plt.savefig('../charts/toxic_roc_1.png')

In [ ]:
rd = pd.DataFrame(results_dict).T
rd = rd.apply(lambda x: round(100*x,1).astype(str) + "%")
rd = rd.sort_values(['auc'],ascending=[False])
rd

In [ ]:
# count_vect.vocabulary_.get(u'chicken')

In [ ]:
# tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
# X_train_tf = tf_transformer.transform(X_train_counts)
# X_train_tf.shape

In [ ]:
# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_train_tfidf.shape

In [ ]:
# clf = MultinomialNB().fit(X_train_tfidf, y)
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)

print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None))])
text_clf.fit(X_train,y_train) 
y_pred = text_clf.predict(X_test)
print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('pipe', KNeighborsClassifier(27))])
pipe.fit(X_train,y_train) 
y_pred = pipe.predict(X_test)
print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred)) 

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('pipe', RandomForestClassifier(max_features='auto', n_estimators=1000))])
pipe.fit(X_train,y_train) 
y_pred = pipe.predict(X_test)
print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('pipe', GradientBoostingClassifier(n_estimators=500))])
pipe.fit(X_train,y_train) 
y_pred = pipe.predict(X_test)
print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
# pipe = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('pipe', XGBClassifier())])
# pipe.fit(X_train,y_train) 
# y_pred = pipe.predict(X_test)
# print(np.mean(y_pred == y_test))
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))

In [ ]:
pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('pipe', SVC(C=10,gamma=0.001,probability=True))])
pipe.fit(X_train,y_train) 
y_pred = pipe.predict(X_test)
print(np.mean(y_pred == y_test))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [ ]:
# print(X_train.shape, X_test.shape, y_train.shape,y_test.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3)}

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [ ]:
gs_clf = gs_clf.fit(X_train, y_train)

In [ ]:
# downsample comments to number of toxic text
from sklearn.utils import resample
df1 = df[['comment_text','toxic']]
X = df1.iloc[:,0]
y = df1.iloc[:,1]
print(X.shape, y.shape)
y[y==0].count()

In [ ]:
X_d, y_d = resample(X[y == 1],y[y == 1],replace=True, n_samples=X[y == 1].shape[0],random_state=42)
print(X_d.shape, y_d.shape)
y_d[y_d==1].count()

In [ ]:
X_bal = np.vstack((X[y==1],X_d))
y_bal = np.hstack((y[y==1],y_d))
print(X_bal.shape,y_bal.shape)

In [ ]:
X_bal = pd.DataFrame(X_bal)
X_bal

In [ ]:
df2 = pd.concat([X_bal, y_bal], axis=1)

In [ ]:
df2.sum(axis=0,numeric_only=True)

In [ ]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(df1.comment_text)
# X = X.toarray()
# count_vect.vocabulary_
# X = df[['word_count','polarity_min','polarity_max','polarity_mean']]
y = df1['toxic']
# y = df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
print(X.shape, y.shape) 

In [ ]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(df.comment_text)
# X = X.toarray()
print(X.shape) 
# count_vect.vocabulary_
# X = df[['word_count','polarity_min','polarity_max','polarity_mean']]
y = df['toxic']
# y = df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [ ]:
X = X.toarray()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_d, y_d = resample(X[y == 0],y[y == 0],replace=True, n_samples=X[y == 1].shape[0],random_state=42)
print(X_d.shape, y_d.shape)

In [ ]:
# downsample comments to number of toxic text
from sklearn.utils import resample
df1 = df[['comment_text','toxic']]
X = df1.iloc[:,0]
y = df1.iloc[:,1]
print(X.shape, y.shape)
y[y==1].count()

In [ ]:
# print(X[y==0].shape, X[y==1].shape)

In [ ]:
# X_imb = np.vstack((X[y == 0], X[y == 1][:40]))

# y_imb = np.hstack((y[y == 0], y[y == 1][:40]))

print(X.shape, y.shape)

In [ ]:
X_d, y_d = resample(X[y == 0],y[y == 0],replace=True, n_samples=X[y == 1].shape[0],random_state=42)
print(X_d.shape, y_d.shape)

In [ ]:
count_vect = CountVectorizer()
X = count_vect.fit_transform(df1.comment_text)
print(X)
y = y_d
# X = X.toarray()
# print(X.shape) 
# count_vect.vocabulary_
# X = df[['word_count','polarity_min','polarity_max','polarity_mean']]
# y = df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [ ]:
# from sklearn.utils import resample

# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# X_train_up, y_train_up = resample(X_train[y_train == 1],y_train[y_train == 1],replace=True,
#                                     n_samples=X_train[y_train == 0].shape[0],random_state=123)

In [ ]:
# print(X_train_up.shape, y_train_up.shape)

In [ ]:
# note that classes are not mutually exclusive, any comment to belong to any of 6 classes
# as such, may need to test each classification separately, unless there is a way to test all together?
# df['rating'] = df['toxic'] + df['severe_toxic'] + df['obscene'] + df['threat'] + df['insult'] + df['identity_hate']
# df['token_clean'] = df['comment_text'].apply(token_clean)
# df['sent_token'] = df['comment_text'].apply(sentence_tokenizer)
# df['polarity_sent_token'] = df['sent_token'].apply(polarity)
# df['word_count'] = df['token_clean'].apply(len)

In [ ]:
# df['polarity_min'] = [x[0] for x in df['polarity_sent_token']]
# df['polarity_max'] = [x[1] for x in df['polarity_sent_token']]
# df['polarity_mean'] = [x[2] for x in df['polarity_sent_token']]
# df.head()

In [ ]:
# df['stemmer'] = df['comment_text'].apply(stemmer)

In [ ]:
df.head()

In [ ]:
# df.groupby('rating').nunique()['id'] # class imbalance issue

In [ ]:
# df = df.sort_values(['rating'],ascending=[False])
# df.head()

In [ ]:
# pipeline
# pipe_lr = make_pipeline(StandardScaler(),
#                          PCA(n_components=2),
#                          LogisticRegression(random_state=1))
# pipe_lr.fit(X_train, y_train)
# y_pred = pipe_lr.predict(X_test)
# print('Test Accuracy: %.3f' % pipe_lr.score(X_test, y_test))

In [ ]:
# https://elitedatascience.com/python-machine-learning-tutorial-scikit-learn
# scaler = StandardScaler(with_mean=False).fit(X_train)
# X_train_s = scaler.transform(X_train)
# X_test_s = scaler.transform(X_test)

In [ ]:
# class DenseTransformer(TransformerMixin):

#     def transform(self, X, y=None, **fit_params):
#         return X.todense()

#     def fit_transform(self, X, y=None, **fit_params):
#         self.fit(X, y, **fit_params)
#         return self.transform(X)

#     def fit(self, X, y=None, **fit_params):
#         return self

In [ ]:
# pipeline = Pipeline([
#      ('vectorizer', CountVectorizer()), 
#      ('to_dense', DenseTransformer()), 
#      ('classifier', RandomForestClassifier())
# ])

In [ ]:
# from sklearn.svm import LinearSVC
# # pipeline = Pipeline([('vectorizer', CountVectorizer()), ('classifier', LinearSVC())])
# predicted = pipeline.predict(X_test)


In [ ]:
# Define models to test
model_list = [['GaussianNB', GaussianNB()], 
                ['BernoulliNB', BernoulliNB()], 
#                 ['MultinomialNB', MultinomialNB()],
                ['DecisionTree', DecisionTreeClassifier(class_weight='balanced')], 
                ['KNN', KNeighborsClassifier(10)], 
                ['RandomForest', RandomForestClassifier(class_weight='balanced')], 
                ['GradientBoost', GradientBoostingClassifier()],
                ['AdaBoost', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=DecisionTreeClassifier(class_weight='balanced'))],
                ['XGBoost', XGBClassifier()],
                ['LogisticRegression', LogisticRegression(class_weight='balanced')],          
                ['SVM', SVC(probability=True, class_weight='balanced')]] # scale data; F1 0.57

model_list_s = ['KNN','LogisticRegression','SVM'] # standardize/normalize data

# Calculate metrics for each model
roc = {}
results_dict = {}
for model in model_list:
    if model[0] in model_list_s:
        X_train = X_train_s
        X_test = X_test_s
    
    model_name = model[0]
    model = model[1]
    
    accuracy = []
    precision_1 = []
    precision_0 = []
    recall_1 = []
    recall_0 = []
    f1_1 = []
    f1_0 = []
    auc = []
        
    # Perform K-Fold CV and calculate metrics for each fold
    kf = KFold(5, random_state=42, shuffle=True) 
    for train_idx, test_idx in kf.split(X, y=y):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy.append(accuracy_score(y_test, y_pred))
        precision_1.append(precision_score(y_test, y_pred ,pos_label=1))
        precision_0.append(precision_score(y_test, y_pred ,pos_label=0))
        recall_1.append(recall_score(y_test, y_pred, pos_label=1))
        recall_0.append(recall_score(y_test, y_pred, pos_label=0))
        f1_1.append(f1_score(y_test, y_pred, pos_label=1))
        f1_0.append(f1_score(y_test, y_pred, pos_label=0))
        auc.append(roc_auc_score(y_test, y_pred))
        
    # Calculate mean metric across K-folds
    mean_accuracy = np.mean(accuracy)
    mean_precision_1 = np.mean(precision_1)
    mean_precision_0 = np.mean(precision_0)
    mean_recall_1 = np.mean(recall_1)
    mean_recall_0 = np.mean(recall_0)
    mean_f1_1 = np.mean(f1_1)
    mean_f1_0 = np.mean(f1_0)
    mean_auc = np.mean(auc)
    
    # Capture TPR and FPR from last fold for plotting
    y_score = model.predict_proba(X_test)[:,1]
    roc[model_name] = roc_curve(y_test, y_score), mean_auc
    results_dict[model_name] = {"accuracy": mean_accuracy, "precision_s": mean_precision_1, "precision_f": mean_precision_0, "recall_s": mean_recall_1, "recall_f": mean_recall_0, "f1_s": mean_f1_1, "f1_f": mean_f1_0, "auc": mean_auc}
    
    # Print formatted results
    print(model)
    print('\t==============================')
    print('\tAccuracy:', mean_accuracy)
    print('\tAUC:', mean_auc)
    print('\n')
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred)) 

In [ ]:
# Plot the ROC curve from the last K-Fold split
fig, ax = plt.subplots(figsize=(10, 10))

# Plot 50-50 Line
ax.plot([0,1],[0,1], ls='--', color='k', label='50-50')

# Plot Classifier ROC Curves
for key, value in roc.items():
    label = '{}, AUC: {}%'.format(key, round(100*value[1],1))
    ax.plot(roc[key][0][0], roc[key][0][1], label=label)
    
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
ax.set_title('ROC Curve - All Models',fontweight='bold',fontsize=15)
ax.legend(loc='best')
plt.savefig('../charts/toxic_roc.png')

In [ ]:
rd = pd.DataFrame(results_dict).T
rd = rd.apply(lambda x: round(100*x,1).astype(str) + "%")
rd = rd.sort_values(['auc'],ascending=[False])
rd

In [ ]:
# cv = CountVectorizer()

# r = pd.SparseDataFrame(cv.fit_transform(text), 
#                        df.index,
#                        cv.get_feature_names(), 
#                        default_fill_value=0)

In [ ]:
# text = df['comment_text'].iloc[0]
# x_back = count_vectorizer(text)
# df1 = pd.DataFrame(x_back,columns=vectorizer.get_feature_names())

In [ ]:
# stop = stopwords.words('english')
# stop += ['.', ',', '(', ')', "'", '"']
# stop = set(stop)

# counter = Counter()

# n = 2
# for doc in df['comment_text']:
#     words = TextBlob(doc).words
#     words = [w for w in words if w not in stop]
#     bigrams = ngrams(words, n)
#     counter += Counter(bigrams)

# for phrase, count in counter.most_common(30):
#     print('%20s %i' % (" ".join(phrase), count))

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

text = ['That is should come to this!', 'This above all: to thine own self be true.', 'Something is rotten in the state of Denmark.']

# # CountVectorizer is a class; so `vectorizer` below represents an instance of that object.
# vectorizer = CountVectorizer(ngram_range=(1,2)) # selects uni and bigrams

# # call `fit` to build the vocabulary
# vectorizer.fit(text)

# # then, use `get_feature_names` to return the tokens
# print(vectorizer.get_feature_names())

# # finally, call `transform` to convert text to a bag of words
# x = vectorizer.transform(text)

In [ ]:
# print('Sparse Matrix')
# # A compressed version; the "sparse" matrix.
# print(type(x))
# print(x)

# print ('Matrix')
# x_back = x.toarray()
# print(type(x_back))
# print(x_back)

In [ ]:
# x_back = df['token_clean'].apply(count_vectorizer)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['comment_text'])
print(X_train_counts.shape)
X_train_counts

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [ ]:
pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

In [ ]:
# x_back = count_vectorizer(text)
# pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

In [ ]:
#### TF: frequency in this document
#### IDF: inverse frequency in the corpus

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
doc_vectors = vectorizer.fit_transform(text)

classes = np.array(['pos']*50 + ['neg']*50)


model = MultinomialNB().fit(doc_vectors, classes)

In [ ]:
# sentences = df['sent_token'].iloc[0]
# whitespace_tokenizer(sentences)

In [ ]:
# tokenizer = TreebankWordTokenizer()
# tokenizer.tokenize(sentences[2])

In [ ]:
# tokenizer = WhitespaceTokenizer()
# tokenizer.tokenize(sentences[2])

In [ ]:
# df.head()

In [ ]:
# df.to_pickle('../data/toxictrain.pkl')

In [ ]:
# RESOURCES
# http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
# https://stackoverflow.com/questions/28384680/scikit-learns-pipeline-a-sparse-matrix-was-passed-but-dense-data-is-required#28384887